In [1]:
import os
import numpy as np
from statistics import mean

import torch
import torchvision

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from scipy import stats

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

PROJECT_PATH = os.getenv('HOME') + '/aiffel/0_felton'
MODEL_PATH = os.path.join(PROJECT_PATH, 'weights')
DATA_PATH = os.path.join(PROJECT_PATH, 'data_ex_total')
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
REJECT_PATH = os.path.join(DATA_PATH, 'reject')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # 여기서 'cuda'가 출력되어야 GPU와 연결이 됩니다

cuda


In [ ]:
# train path image 확인 각폴더당 1개
for dirpath, dirnames, filenames in os.walk(TRAIN_PATH):
    for i, filename in enumerate(filenames):
        print(os.path.join(dirpath, filename)) # 파일이름을 출력합니다
        image = Image.open(os.path.join(dirpath, filename), 'r')
        print(f'size: ({image.width}, {image.height}, {image.getbands()})') # 이미지 정보를 출력합니다
        plt.imshow(image)
        plt.show()
        break # 폴더마다 1장만 출력합니다

In [4]:
# data 전처리 함수 만들기 resize 224, 224 -> resNet 사용 shape조절

# train data image 증강
# random H-flip, V-flip image 증강
# 색깔 바꾸기 colorjitter 

def create_dataloader(path, batch_size, istrain):
    nearest_mode = torchvision.transforms.InterpolationMode.NEAREST
    normalize = torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
    train_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ColorJitter(),
        torchvision.transforms.ToTensor(),
        normalize
    ])

    test_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
        torchvision.transforms.ToTensor(),
        normalize
    ])
    
    if istrain:
        data = torchvision.datasets.ImageFolder(path, transform=train_transformer)
        dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
        
    else:
        data = torchvision.datasets.ImageFolder(path, transform=test_transformer)
        dataloader = torch.utils.data.DataLoader(data, shuffle=False)

    return dataloader, data


In [5]:
# 위에서 만든 함수로 데이터셋 준비
# train dataset
# batch_size = 32

BATCH_SIZE = 32

train_loader, _train_data = create_dataloader(TRAIN_PATH, BATCH_SIZE, True)
target_class_num = len(os.listdir(os.path.join(TRAIN_PATH)))

print('target_class_num: ', target_class_num)
print('train: ', _train_data.class_to_idx)

target_class_num:  2
train:  {'clean': 0, 'dirt': 1}


In [6]:
# 각 클래스별 이미지 개수 확인
for dirpath, dirnames, filenames in os.walk(TRAIN_PATH):
    print(f'{dirpath} : {len(filenames)}')

/aiffel/aiffel/0_felton/data_ex_total/train : 0
/aiffel/aiffel/0_felton/data_ex_total/train/dirt : 3926
/aiffel/aiffel/0_felton/data_ex_total/train/clean : 3761


In [7]:
# test data set 준비
# shuffle = False
test_loader, _test_data = create_dataloader(TEST_PATH, BATCH_SIZE, False)
print('test: ', _test_data.class_to_idx)

test:  {'clean': 0, 'dirt': 1}


In [8]:
# 개수 확인
for dirpath, dirnames, filenames in os.walk(TEST_PATH):
    print(f'{dirpath} : {len(filenames)}')

/aiffel/aiffel/0_felton/data_ex_total/test : 0
/aiffel/aiffel/0_felton/data_ex_total/test/dirt : 600
/aiffel/aiffel/0_felton/data_ex_total/test/clean : 600


### model 훈련

In [9]:
# Metric 함수
def calculate_metrics(trues, preds):
    accuracy = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='macro')
    precision = precision_score(trues, preds, average='macro')
    recall = recall_score(trues, preds, average='macro')
    return accuracy, f1, precision, recall

In [10]:
# train 함수
def train(dataloader, net, learning_rate, weight_decay_level, device):
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        net.parameters(),
        lr = learning_rate, 
        weight_decay = weight_decay_level
    )

    net.train()

    train_losses = list()
    train_preds = list()
    train_trues = list()
    
    train_loss = 0
    correct = 0
    total = 0

    for idx, (img, label) in enumerate(dataloader):

        img = img.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()

        out = net(img)

        _, pred = torch.max(out, 1)
        loss = criterion(out, label)

        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = out.max(1)
        
        total += label.size(0)
        correct += predicted.eq(label).sum().item() 

        train_losses.append(loss.item())
        train_trues.extend(label.view(-1).cpu().numpy().tolist())
        train_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(train_trues, train_preds)

    print('\n''====== Training Metrics ======')
    print('Loss: ', mean(train_losses))
    print('Acc: ', acc)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', rec)
    print(confusion_matrix(train_trues, train_preds))

    return net, acc, f1, prec, rec

In [11]:
# test 함수
def test(dataloader, net, device):

    criterion = torch.nn.CrossEntropyLoss()
    
    net.eval()
    test_losses = list()
    test_trues = list()
    test_preds = list()
    
    with torch.no_grad():
        for idx, (img, label) in enumerate(dataloader):

            img = img.to(device)
            label = label.to(device)

            out = net(img)

            _, pred = torch.max(out, 1)
            loss = criterion(out, label)

            test_losses.append(loss.item())
            test_trues.extend(label.view(-1).cpu().numpy().tolist())
            test_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(test_trues, test_preds)

    print('====== Test Metrics ======')
    print('Test Loss: ', mean(test_losses))
    print('Test Acc: ', acc)
    print('Test F1: ', f1)
    print('Test Precision: ', prec)
    print('Test Recall: ', rec)
    print(confusion_matrix(test_trues, test_preds))

    return net, acc, f1, prec, rec

In [12]:
# 학습용 함수
# train -> test -> save 반복
# 가장 좋은 accuracy 를 저장

def train_classifier(net, train_loader, test_loader, n_epochs, learning_rate, weight_decay, device):
    best_test_acc = 0
    
    model_save_path = None
    model_save_base = 'weights'
    if not os.path.exists(model_save_base):
        os.makedirs(model_save_base)
    
    print('>> Start Training Model!')
    for epoch in range(n_epochs):
        
        print('> epoch: ', epoch)

        net, _, _, _, _ = train(train_loader, net, learning_rate, weight_decay, device)
        net, test_acc, _, _, _  = test(test_loader, net, device)

        if test_acc > best_test_acc:

            best_test_acc = test_acc
            test_acc_str = '%.5f' % test_acc

            print('[Notification] Best Model Updated!')
            model_save_path = os.path.join(model_save_base, 'classifier_acc_' + str(test_acc_str) + '.pth') 
            torch.save(net.state_dict(), model_save_path)
                
    return model_save_path

In [13]:
# 모델 가져오기
# 사용모델 resNet 50, pretrained = True
# pretraine 된 모델로 전이 학습예정
net = torchvision.models.resnet50(pretrained=True)
net.fc = torch.nn.Linear(
    net.fc.in_features,
    target_class_num
)

net.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /aiffel/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### 모델 학습 시키기

In [14]:
# 1 epoch  시험 
EPOCHS = 1
LEARNING_RATE = 0.005
WEIGHT_DECAY = 0.0005

saved_weight_path = train_classifier(net, train_loader, test_loader, EPOCHS, LEARNING_RATE, WEIGHT_DECAY, device)

>> Start Training Model!
> epoch:  0

====== Training Metrics ======
Loss:  0.27876008037744376
Acc:  0.8850006504488097
F1:  0.8849250717513424
Precision:  0.8850226249310439
Recall:  0.8848582449877371
[[3303  458]
 [ 426 3500]]
====== Test Metrics ======
Test Loss:  0.6127794181946696
Test Acc:  0.7408333333333333
Test F1:  0.7320718983481357
Test Precision:  0.7770755901837902
Test Recall:  0.7408333333333333
[[336 264]
 [ 47 553]]
[Notification] Best Model Updated!


In [15]:
# 20 epoch  시험 
EPOCHS = 20
LEARNING_RATE = 0.005
WEIGHT_DECAY = 0.0005

saved_weight_path = train_classifier(net, train_loader, test_loader, EPOCHS, LEARNING_RATE, WEIGHT_DECAY, device)

>> Start Training Model!
> epoch:  0

====== Training Metrics ======
Loss:  0.10919307575163072
Acc:  0.9652660335631585
F1:  0.96523326373478
Precision:  0.9655800115669408
Recall:  0.965051674537844
[[3592  169]
 [  98 3828]]
====== Test Metrics ======
Test Loss:  0.015413598019474162
Test Acc:  0.995
Test F1:  0.9949999861110724
Test Precision:  0.9950055000611118
Test Recall:  0.995
[[598   2]
 [  4 596]]
[Notification] Best Model Updated!
> epoch:  1

====== Training Metrics ======
Loss:  0.06023066104640914
Acc:  0.9802263561857681
F1:  0.9802136234498902
Precision:  0.9803237312392902
Recall:  0.9801334323376509
[[3670   91]
 [  61 3865]]
====== Test Metrics ======
Test Loss:  0.2928140545806552
Test Acc:  0.8691666666666666
Test F1:  0.866888142941221
Test Precision:  0.8963011889035667
Test Recall:  0.8691666666666666
[[443 157]
 [  0 600]]
> epoch:  2

====== Training Metrics ======
Loss:  0.04062240491273399
Acc:  0.9876414726161051
F1:  0.9876352820930645
Precision:  0.9876